In [67]:
import numpy as np
import pandas as pd

In [68]:
#Read Data
cgm=pd.read_csv('CGMData.csv',low_memory=False,usecols=['Date','Time','Sensor Glucose (mg/dL)'])
insulin=pd.read_csv('InsulinData.csv',low_memory=False, usecols=['Date','Time','Alarm'])

In [69]:
#Create Data Time Stamp columns
cgm['data_time_stamp']=pd.to_datetime(cgm['Date'] + ' ' + cgm['Time'])
insulin['data_time_stamp']=pd.to_datetime(insulin['Date'] + ' ' + insulin['Time'])

In [70]:
#Cleaning Data
cgm.sort_values(by=['data_time_stamp'], ascending = True, inplace = True)
insulin.sort_values(by=['data_time_stamp'], ascending = True, inplace = True) 
nan_dates=cgm[cgm['Sensor Glucose (mg/dL)'].isna()]['Date'].unique()
cgm=cgm.set_index('Date').drop(index=nan_dates).reset_index()

cgm_dates=cgm.groupby('Date')['Sensor Glucose (mg/dL)'].count().where(lambda x:x>0.8*288).dropna().index.tolist()
cgm=cgm.loc[cgm['Date'].isin(cgm_dates)]

In [71]:
auto_mode_start=insulin.loc[insulin['Alarm']=='AUTO MODE ACTIVE PLGM OFF'].iloc[0]['data_time_stamp']
auto_mode_data=cgm.loc[cgm['data_time_stamp']>=auto_mode_start]
manual_mode_data=cgm.loc[cgm['data_time_stamp']<auto_mode_start]
cgm.set_index('data_time_stamp', inplace = True)
auto_mode_data.set_index('data_time_stamp', inplace = True)
manual_mode_data.set_index('data_time_stamp', inplace = True)

In [72]:
#Part 1: Percent Time in Hyperglycemia
hyperglycemia_wholeday_automode=(auto_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hyperglycemia_daytime_automode=(auto_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hyperglycemia_overnight_automode=(auto_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

hyperglycemia_wholeday_manualmode=(manual_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hyperglycemia_daytime_manualmode=(manual_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hyperglycemia_overnight_manualmode=(manual_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

In [73]:
#Part 2: Percent time in Hyperglycemia Critical
hyperglycemia_critical_wholeday_automode=(auto_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hyperglycemia_critical_daytime_automode=(auto_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hyperglycemia_critical_overnight_automode=(auto_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

hyperglycemia_critical_wholeday_manualmode=(manual_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hyperglycemia_critical_daytime_manualmode=(manual_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hyperglycemia_critical_overnight_manualmode=(manual_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

In [74]:
#Part 3: Percent time in Range 1
range_wholeday_automode=(auto_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(auto_mode_data['Sensor Glucose (mg/dL)']>=70) & (auto_mode_data['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
range_daytime_automode=(auto_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(auto_mode_data['Sensor Glucose (mg/dL)']>=70) & (auto_mode_data['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
range_overnight_automode=(auto_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(auto_mode_data['Sensor Glucose (mg/dL)']>=70) & (auto_mode_data['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

range_wholeday_manualmode=(manual_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(manual_mode_data['Sensor Glucose (mg/dL)']>=70) & (manual_mode_data['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
range_daytime_manualmode=(manual_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(manual_mode_data['Sensor Glucose (mg/dL)']>=70) & (manual_mode_data['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
range_overnight_manualmode=(manual_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(manual_mode_data['Sensor Glucose (mg/dL)']>=70) & (manual_mode_data['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

In [75]:
#Part 4: Percent time in Range Secondary
range_secondary_wholeday_automode=(auto_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(auto_mode_data['Sensor Glucose (mg/dL)']>=70) & (auto_mode_data['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
range_secondary_daytime_automode=(auto_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(auto_mode_data['Sensor Glucose (mg/dL)']>=70) & (auto_mode_data['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
range_secondary_overnight_automode=(auto_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(auto_mode_data['Sensor Glucose (mg/dL)']>=70) & (auto_mode_data['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

range_secondary_wholeday_manualmode=(manual_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(manual_mode_data['Sensor Glucose (mg/dL)']>=70) & (manual_mode_data['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
range_secondary_daytime_manualmode=(manual_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(manual_mode_data['Sensor Glucose (mg/dL)']>=70) & (manual_mode_data['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
range_secondary_overnight_manualmode=(manual_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[(manual_mode_data['Sensor Glucose (mg/dL)']>=70) & (manual_mode_data['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

In [76]:
#Part 5: Percent time in Hypoglycemia
hypoglycemia_lv1_wholeday_automode=(auto_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hypoglycemia_lv1_daytime_automode=(auto_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hypoglycemia_lv1_overnight_automode=(auto_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

hypoglycemia_lv1_wholeday_manualmode=(manual_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hypoglycemia_lv1_daytime_manualmode=(manual_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hypoglycemia_lv1_overnight_manualmode=(manual_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

In [77]:
#Part 3: Percent time in Hyploglycemia Level 2
hypoglycemia_lv2_wholeday_automode=(auto_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hypoglycemia_lv2_daytime_automode=(auto_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hypoglycemia_lv2_overnight_automode=(auto_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[auto_mode_data['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

hypoglycemia_lv2_wholeday_manualmode=(manual_mode_data.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hypoglycemia_lv2_daytime_manualmode=(manual_mode_data.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)
hypoglycemia_lv2_overnight_manualmode=(manual_mode_data.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']].loc[manual_mode_data['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100)

In [78]:
#Storing Means in CSV File(2 X 18) 
result_df = pd.DataFrame({'Percent Time in HyperGlycemia Daytime': [hyperglycemia_daytime_manualmode.mean(axis=0),  hyperglycemia_daytime_automode.mean(axis=0)], 'Percent Time in HyperGlycemia Overnight': [hyperglycemia_overnight_manualmode.mean(axis=0), hyperglycemia_overnight_automode.mean(axis=0)], 'Percent Time in HyperGlycemia Wholeday': [hyperglycemia_wholeday_manualmode.mean(axis=0), hyperglycemia_wholeday_automode.mean(axis=0)], 'Percent time in HyperGlycemia Critical Daytime': [hyperglycemia_critical_daytime_manualmode.mean(axis=0), hyperglycemia_critical_daytime_automode.mean(axis=0)], 'Percent time in HyperGlycemia Critical Overnight':[hyperglycemia_critical_overnight_manualmode.mean(axis=0), hyperglycemia_critical_overnight_automode.mean(axis=0)], 'Percent time in HyperGlycemia Critical Wholeday':[hyperglycemia_critical_wholeday_manualmode.mean(axis=0), hyperglycemia_critical_wholeday_automode.mean(axis=0)], 'Percent time in Range Daytime': [range_daytime_manualmode.mean(axis=0), range_daytime_automode.mean(axis=0)], 'Percent time in Range Overnight':[range_overnight_manualmode.mean(axis=0), range_overnight_automode.mean(axis=0)], 'Percent time in Range Wholeday':[range_wholeday_manualmode.mean(axis=0), range_wholeday_automode.mean(axis=0)], 'Percent time in Range Secondary Daytime': [range_secondary_daytime_manualmode.mean(axis=0), range_secondary_daytime_automode.mean(axis=0)], 'Percent time in Range Secondary Overnight':[range_secondary_overnight_manualmode.mean(axis=0), range_secondary_overnight_automode.mean(axis=0)], 'Percent time in Range Secondary Wholeday':[range_secondary_wholeday_manualmode.mean(axis=0), range_secondary_wholeday_automode.mean(axis=0)], 'Percent time in HypoGlycemia Daytime': [hypoglycemia_lv1_daytime_manualmode.mean(axis=0), hypoglycemia_lv1_daytime_automode.mean(axis=0)], 'Percent time in HypoGlycemia Overnight':[hypoglycemia_lv1_overnight_manualmode.mean(axis=0), hypoglycemia_lv1_overnight_automode.mean(axis=0)], 'Percent time in HypoGlycemia Wholeday':[hypoglycemia_lv1_wholeday_manualmode.mean(axis=0), hypoglycemia_lv1_wholeday_automode.mean(axis=0)], 'Percent time in HypoGlycemia Level 2 Daytime': [hypoglycemia_lv2_daytime_manualmode.mean(axis=0), hypoglycemia_lv2_daytime_automode.mean(axis=0)], 'Percent time in HypoGlycemia Level 2 Overnight':[np.nan_to_num(hypoglycemia_lv2_overnight_manualmode.mean(axis=0)), np.nan_to_num(hypoglycemia_lv2_overnight_automode.mean(axis=0))], 'Percent time in HypoGlycemia Level 2 Wholeday':[hypoglycemia_lv2_wholeday_manualmode.mean(axis=0), hypoglycemia_lv2_wholeday_automode.mean(axis=0)]})

In [79]:
result_df.to_csv('Results.csv',header=False,index=False)

In [80]:
print(result_df)

   Percent Time in HyperGlycemia Daytime  \
0                              25.347222   
1                              20.193142   

   Percent Time in HyperGlycemia Overnight  \
0                                11.458333   
1                                 6.543803   

   Percent Time in HyperGlycemia Wholeday  \
0                               27.152778   
1                               22.851563   

   Percent time in HyperGlycemia Critical Daytime  \
0                                       17.187500   
1                                        8.002646   

   Percent time in HyperGlycemia Critical Overnight  \
0                                         10.069444   
1                                          4.745370   

   Percent time in HyperGlycemia Critical Wholeday  \
0                                        22.222222   
1                                         8.680556   

   Percent time in Range Daytime  Percent time in Range Overnight  \
0                      37.708333  